<center> <h1> 机器作诗进阶：RNN </h1> </center>  

# 逻辑回归（CNN）模型写诗，缺点？

> 静夜思：床前明月光，疑似地上霜，举头望明月，低头思故乡

> 输入长度必须固定（例如：X=明，月，光）！问：能否长度任意？

> 输入没有记忆（例如：X=前，明，月；床？去哪里了？）！问：能否对历史有记忆！

$$P\Big (X_{t}=光\Big)=f\Big(X_{t-1}=月,X_{t-2}=明,X_{t-3}=床\Big)$$



# 一个更为合理的建模方式

> 对于“序列数据”中信息的充分提取，需要我们将“历史”的作用不断“传递”

<img src="./graph/f2-2.PNG" alt="drawing" width="400"/>


# 状态空间模型

> 静夜思：床前明月光，疑似地上霜，举头望明月，低头思故乡


$$Z_{t+1}=h\Big(X_{t}=故,Z_{t}\Big).$$
$$P\Big (X_{t+1}=乡\Big)=f\Big(Z_{t+1}\Big)$$




# RNN：更好的处理“序列数据”

> RNN（Recurrent Neural Network），也叫循环神经网络，是一种专门处理“文本序列数据”的方法。

> 核心思想：通过将历史信息不断保留与传递，而保留与传递的载体就是状态$Z_t$。

> RNN最早被认知科学与计算神经科学的研究人员提出并应用，后来被广泛应用于研究序列数据，下图展示了一些早期的RNN相关文献。


<img src="./graph/f2-3.png" alt="drawing" width="750"/>

# RNN结构

> 当前时刻的数据 + 上一时刻的状态 = 当前时刻的状态

> 每一次神经网络的判断过程之后，都会把信息传给下一次判断过程，就类似于我们人脑的思考理解过程


<img src="./graph/RNN.png" alt="drawing" width="650"/>

> 非线性变换：$Z_{t+1}=f(W_1X_t,W_2Z_t)$。问题：（1）可以考虑什么样的非线性变幻？（2）消耗多少个参数？

# 数据介绍

data/poems_clean.txt

格式：标题:诗（每句之间以空格隔开）

例子：

    静夜思:床前明月光 疑是地上霜 举头望明月 低头思故乡  
    春望:国破山河在 城春草木深 感时花溅泪 恨别鸟惊心 烽火连三月 家书抵万金 白头搔更短 浑欲不胜簪 

# 数据读入与展示

In [1]:
import string
import numpy as np

f = open('data/poems_clean.txt', "r", encoding='utf-8')
poems = []
for line in f.readlines():
    title, poem = line.split(':')
    poem = poem.replace(' ', '') #将空格去掉
    poem = poem.replace('\n', '') #将换行符去掉
    poems.append(list(poem))
    
print(poems[0][:])

['寒', '随', '穷', '律', '变', '春', '逐', '鸟', '声', '开', '初', '风', '飘', '带', '柳', '晚', '雪', '间', '花', '梅', '碧', '林', '青', '旧', '竹', '绿', '沼', '翠', '新', '苔', '芝', '田', '初', '雁', '去', '绮', '树', '巧', '莺', '来']


# 数据整理：文字编码

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(poems)
poems_digit = tokenizer.texts_to_sequences(poems)
vocab_size = len(tokenizer.word_index) + 1 #加上停止词0
vocab_size #有多少个不同的字


Using TensorFlow backend.


5546

# 数据整理

> 由于每首诗的长度不一致，为了将所有的诗放在一个统一的$m\times n$维数组中，我们需要在较短诗的末尾用0进行补齐

<img src="./graph/f2-4.PNG" alt="drawing" width="750"/>

## （1）补全数据

In [3]:
#为了将所有的诗放在一个M*N的np.array中，将每一首诗补0到同样的长度
poems_digit = pad_sequences(poems_digit, maxlen=50, padding='post')
print("原始诗歌")
print(poems[3864])
print("\n")
print("编码+补全后的结果")
print(poems_digit[3864])

原始诗歌
['床', '前', '明', '月', '光', '疑', '是', '地', '上', '霜', '举', '头', '望', '明', '月', '低', '头', '思', '故', '乡']


编码+补全后的结果
[532  72  53  13 140 429  44 113  15 202 688 128 106  53  13 502 128  75
 134 169   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


## （2）对齐$X$和$Y$


> 将每首诗的前一个字作为$X$，后一个字作为$Y$，进行数据对齐

In [4]:
X = poems_digit[:, :-1]
Y = poems_digit[:, 1:]

print(poems_digit.shape)
print(X.shape)
print(Y.shape)

print("X示例", "\t", "Y示例")

for i in range(10):
    print(X[0][i], "\t", Y[0][i])
    
print("...", "\t", "...")

(24117, 50)
(24117, 49)
(24117, 49)
X示例 	 Y示例
42 	 180
180 	 401
401 	 1143
1143 	 671
671 	 9
9 	 331
331 	 130
130 	 58
58 	 84
84 	 177
... 	 ...


## （3）把$Y$变成One-Hot向量


In [5]:
print(vocab_size)
from keras.utils import to_categorical
Y = to_categorical(Y, num_classes=vocab_size)
print(Y.shape)

5546
(24117, 49, 5546)


# 确定空间维度

In [6]:
from keras.models import Model
from keras.layers import Input, SimpleRNN, Dense, Embedding, Activation, BatchNormalization
embedding_size = 64
hidden_size = 128

# 构建RNN的模型

In [7]:
inp = Input(shape=(49,))

# Encoder
x = Embedding(vocab_size, embedding_size, mask_zero=True)(inp)
x = SimpleRNN(hidden_size,return_sequences=True)(x)

# prediction
x = Dense(vocab_size)(x)
pred = Activation('softmax')(x)

model = Model(inp, pred)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 49)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 49, 64)            354944    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 49, 128)           24704     
_________________________________________________________________
dense_1 (Dense)              (None, 49, 5546)          715434    
_________________________________________________________________
activation_1 (Activation)    (None, 49, 5546)          0         
Total params: 1,095,082
Trainable params: 1,095,082
Non-trainable params: 0
_________________________________________________________________


# 参数个数

> vocab_size=5546; embedding_size=64; hidden_size=128

> Embedding: 5546*64 = 354944

> RNN：128*64 +128*128+128 = 24704 

> Dense： 5546*128+5546=715434

# 模型训练

In [9]:
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.fit(X, Y, epochs=2, batch_size=128, validation_split=0.2)

# 应用模型作诗

In [ ]:
poem_incomplete = '熊****大****很****帅****'
poem_index = []
poem_text = ''
for i in range(len(poem_incomplete)):
    current_word = poem_incomplete[i]
    
    if  current_word != '*':
        index = tokenizer.word_index[current_word]
        
    else:
        x = np.expand_dims(poem_index, axis=0)
        x = pad_sequences(x, maxlen=49, padding='post')
        y = model.predict(x)[0, i]
        
        y[0] = 0            #去掉停止词
        index = y.argmax()
        current_word = tokenizer.index_word[index]
        
        


    poem_index.append(index)
    poem_text = poem_text + current_word
        
poem_text = poem_text[0:]
print(poem_text[0:5])
print(poem_text[5:10])
print(poem_text[10:15])
print(poem_text[15:20])

# 思考问题：RNN vs. 时间序列模型